## Metrics: Calculating_02: tf.metrics() intro


<font color=red>TODO: add links and information on the series here </font>

#### series information
- previous: calculating accuracy [video]() [notebook]()
- next:  [video]() [notebook]()

#### Related series
- metrics [video]() [notebook]()

#### Resources:
 - [official tensorflow documentation](https://www.tensorflow.org/api_docs/python/tf/metrics)

In [1]:
import tensorflow as tf
import numpy as np
import os

# Helper to make the output consistent
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
# supress most messages (display only error messages)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# reset graph
reset_graph()

In [2]:
x = tf.placeholder(tf.int32, [10])
y = tf.placeholder(tf.int32, [10])

## Use tf.metrics()
Using;
```
tf.variable_scope()
tf...get_variables()
tf.variables_initializer()
```

<font color=red>TODO: add links and information </font>

tf.variable_scope crates a scope ------- , then, using tf...get_variables() we create an opperation to initialize (and reset) the specfied variable.

In [3]:
with tf.variable_scope("accuracy_metrics") as scope:
    acc, acc_op = tf.metrics.accuracy(labels=y, predictions=x)
    m_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
    acc_reset_op = tf.variables_initializer(m_vars)

#### Include previous method for comparison

In [4]:
# calculate ratio of correct predictions
correct_prediction = tf.equal(x, y)
batch_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [5]:
# fake data
y_pred_cls = np.array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1])
y_true_cls = np.array([0, 0, 1, 0, 0, 1, 0, 0, 0, 0])

#### Initialize global and local variables
<font color=red>TODO: add links and information on the series here </font>
```
tf.local_variables_initializer()
```
We need to use a local initializer to initialize the metrics in tf.metrics()

In [6]:
# initializers, global and local
init_global = tf.global_variables_initializer()
init_local = tf.local_variables_initializer()

In [7]:
# initialize the session and global and local vars
# note: we will want to close the session at the end `sess.close()`
sess = tf.Session()
init_global.run(session=sess)
init_local.run(session=sess)

## Calculate metrics

### Calculate accuracy

In [8]:
v = sess.run([batch_acc, acc, acc_op], feed_dict={x: y_pred_cls,
                                                  y: y_true_cls})
print("batch_acc: {:.2f}% acc: {:.2f}%, acc_op: {:.2f}%".format(v[0]*100, v[1]*100, v[2]*100))

batch_acc: 40.00% acc: 0.00%, acc_op: 40.00%


#### Examining the results
```
batch acc: 40.00%
acc: 0.00%
acc_op: 40.00%
```
The manual calculation of accuracy (as shown in the previous notebook|video) `batch_acc` produces results as expected.  The `acc` result may not be expected though as it returns **0%** when we know the accuracy should be **40%** and the `acc_op` returns the expected **40%**.

What happens if we run these operations again with different data?

In [9]:
# 80% are correct (matching)
y_pred_cls = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1])
y_true_cls = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

v = sess.run([batch_acc, acc, acc_op], feed_dict={x: y_pred_cls,
                                                  y: y_true_cls})
print("batch_acc: {:.2f}% acc: {:.2f}%, acc_op: {:.2f}%".format(v[0]*100, v[1]*100, v[2]*100))

batch_acc: 80.00% acc: 40.00%, acc_op: 60.00%


### Um.....
```
batch acc: 60.00%
acc: 40.00%
acc_op: 50.00%
```
What just happened?

This might make sense to you -- if so, great! If not, I bet you can figure it out pretty quickly.

We expected our value to be **60%** and that is what our `batch_acc` returned.  But acc returned **40%**, which, if you remember was our previous data's accuracy and our `acc_op` returned **50%**.

If you read the documentation -- which, I'm sure you did, right? ... right? (here's a [link](https://www.tensorflow.org/api_docs/python/tf/metrics/accuracy) again just in case) -- then you'll find that `tf.metrics.accuracy()` is pretty clever under the hood. The function creates and updates two local variables (`total` and `count`) that are used to calculate the metrics.

In [10]:
v = sess.run([batch_acc, acc, acc_op], feed_dict={x: y_pred_cls,
                                                  y: y_true_cls})
print("batch_acc: {:.2f}% acc: {:.2f}%, acc_op: {:.2f}%".format(v[0]*100, v[1]*100, v[2]*100))

batch_acc: 80.00% acc: 60.00%, acc_op: 66.67%


```
batch acc: 80.00%
acc: 60.00%
acc_op: 66.67%
```

We expected **80%** and that's what `batch_acc` returned. `acc` is returning the previous `accuracy` value **60.00%** and `acc_op` is returning **66.67%**.


### What's all this 66.67% about?
so let's think about what data we've fed into our session.
1. 4/10
1. 8/10
1. 8/10

<font color=red>TODO: add equation </font>

What is the running accuracy of these values? (4+8+8)/(10+10+10) = 20/30 = **66.67%**

So `acc_op` is returning the current running accuracy for us.  Great, but now what is `acc` doing?  It's still returning the previous running accuracy?

In [11]:
# What if I run the opperations in a different order?
v = sess.run([batch_acc, acc_op, acc], feed_dict={x: y_pred_cls,
                                                  y: y_true_cls})
print("batch_acc: {:.2f}% acc_op: {:.2f}%, acc: {:.2f}%".format(v[0]*100, v[1]*100, v[2]*100))

batch_acc: 80.00% acc_op: 70.00%, acc: 66.67%


Nothing intersesting -- we're still seeing the same trend in results.

What if I run them in the different order than before, but instead, run them one at a time?

_note: I'm using a different call here. rather than call sess.run() and pass the operation, I'm `eval()`uating the operation with the `session=`to our `sess`._

In [17]:
print(acc.eval(feed_dict={x: y_pred_cls, y: y_true_cls}, session=sess))

0.7


In [18]:
print(acc_op.eval(feed_dict={x: y_pred_cls, y: y_true_cls}, session=sess))

0.72


In [19]:
print(acc.eval(feed_dict={x: y_pred_cls, y: y_true_cls}, session=sess))

0.72


In [20]:
print(acc.eval(feed_dict={x: y_pred_cls, y: y_true_cls}, session=sess))

0.72


In [21]:
print(acc.eval(feed_dict={x: y_pred_cls, y: y_true_cls}, session=sess))

0.72


In [22]:
print(acc_op.eval(feed_dict={x: y_pred_cls, y: y_true_cls}, session=sess))

0.733333


In [23]:
print(acc_op.eval(feed_dict={x: y_pred_cls, y: y_true_cls}, session=sess))

0.742857


In [24]:
print(acc.eval(feed_dict={x: y_pred_cls, y: y_true_cls}, session=sess))

0.742857


In [25]:
print(acc.eval(feed_dict={x: y_pred_cls, y: y_true_cls}, session=sess))

0.742857


### See it yet?
Should I run a couple more? Only kidding, I'm done.

The point is this, the `acc_op` is updating the local variables and then returning the metric and `acc` is only displaying them.  Pretty cool, right?

In [26]:
# close the session
sess.close()

# Take home

In our exmples, and as well see later;
- `tf.metrics().xxxx()` keeps track of the overall metric value, not only the current batch
- local vars (`total` and `count`) need to be initialized
    - `tf.local_variables_initializer()`
- `acc` returned the current metric value
- `acc_op` updated the local variables then returned the value


## Next steps
- checkout the related [exercises]()
- additional common tf.metrics() [video]()|[notebook]()
- how to better control the metrics [video]()|[notebook]()

<font color=red>TODO: Insert image of accuracy calculation here </font>

Next, we'll [use and discuss more functions]() from the tf.metrics metrics package and then we'll look at how to better [control these metrics]().

### Calculate other metrics

NameError: name 'documentation' is not defined

### Confusion matrix